In [11]:
from HPCv3.Simulator.Simulator import Simulator, run_simulation
from HPCv3.RJMS.RJMS import RJMS

from datetime import datetime

In [20]:
# Path to workload JSON file
workload_path = "workloads/generated.json" 
# Path to platform JSON file
platform_path = "platforms/generated.json"
# Path to output folder
output_path = "results/generated"
# Scheduling algorithm name
algorithm = "fcfs"
# Simulation timeout in seconds (optional)
timeout = None
# Time in format YYYY-MM-DD HH:MM:SS (optional). If not provided, starts at 0.
start_time = None

In [ ]:
if start_time is not None:
    start_time = int(datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S").timestamp())
    human_readable = True
else:
    start_time = 0
    human_readable = False

simulator = Simulator(workload_path, platform_path, start_time)
rjms = RJMS(platform_path, algorithm, start_time, timeout)
run_simulation(simulator, rjms, human_readable, output_path)

[simulator send message to RJMS] {'now': 0, 'event_list': [{'timestamp': 0, 'events': [{'job_id': 1, 'res': 5, 'subtime': 0, 'walltime': 51, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 2, 'res': 4, 'subtime': 0, 'walltime': 51, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 3, 'res': 4, 'subtime': 0, 'walltime': 50, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 4, 'res': 4, 'subtime': 0, 'walltime': 50, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 5, 'res': 3, 'subtime': 0, 'walltime': 52, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 6, 'res': 5, 'subtime': 0, 'walltime': 54, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 7, 'res': 5, 'subtime': 0, 'walltime': 51, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 8, 'res': 4, 'subtime': 0, 'walltime': 47, 'profile': '100', 'user_id': 0, 'type': 'arrival'}, {'job_id': 9, 'res': 4, 'subtime': 0, 'walltime': 50, 'profile': 